## View original data

View original data provided by telegram

In [16]:
import pandas as pd
from tqdm import tqdm_notebook
import json

In [32]:
file = 'dc0202'

with open(f'telegram_data/{file}-input.txt', 'r', encoding='utf-8') as f:
    input_lines = f.read().split('\n')

with open(f'telegram_data/{file}-output.txt', 'r', encoding='utf-8') as f:
    output_lines = f.read().split('\n')
    

data = {
    "title": [],
    "description": [],
    "recent_posts": [],
    "language": []
}

for line in tqdm_notebook(input_lines):
    try:
        line = json.loads(line)
    except:
        pass
    
    if line != '':
        data['title'].append(line['title'])
        data['description'].append(line['description'])
        data['recent_posts'].append('\n'.join(line['recent_posts']))
        
for line in tqdm_notebook(output_lines):
    try:
        line = json.loads(line)
    except:
        pass
    
    if line != '':
        data['language'].append(line['lang_code'])
        
data = pd.DataFrame(data)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


## tgstat2octoparse

Parse tgstat links to octoparse links format

In [405]:
import json
import pandas as pd

CATEGORY = 'Hobbies & Activities'
print(CATEGORY)

with open("data/category.txt", 'r', encoding='utf-8-sig') as f:
    data = json.loads(f.read())
    
with open("data/channelsEN.json", 'r', encoding='utf-8') as f:
    channels = json.loads(f.read())
    
# channels[CATEGORY] = []
# with open("data/channelsEN.json", 'w+', encoding='utf-8') as f:
#     json.dump(channels, f)

channels_data = []
for i, item in enumerate(data['items']['list']):
    channels_data.append(item['channelIdCode'])
channels[CATEGORY] += channels_data

with open("data/channelsEN.json", 'w+', encoding='utf-8') as f:
    json.dump(channels, f)

Hobbies & Activities


In [39]:
for i, item in enumerate(channels.keys()):
    print(i, item, len(channels[item]))

0 Art & Design 340
1 Bets & Gambling 385
2 Books 312
3 Business & Entrepreneurship 571
4 Cars & Other Vehicles 193
5 Celebrities & Lifestyle 300
6 Cryptocurrencies 500
7 Culture & Events 252
8 Curious Facts 376
9 Directories of Channels & Bots 32
10 Economy & Finance 358
11 Education 300
12 Erotic Content 300
13 Fashion & Beauty 160
14 Fitness 64
15 Food & Cooking 78
16 Foreign Languages 200
17 Health & Medicine 200
18 History 230
19 Hobbies & Activities 30
20 Home & Architecture 29
21 Humor & Memes 300
22 Investments 271
23 Job Listings 116
24 Kids & Parenting 29
25 Marketing & PR 300
26 Motivation & Self-Development 300
27 Movies 300
28 Music 300
29 Offers & Promotions 790
30 Pets 109
31 Politics & Incidents 300
32 Psychology & Relationships 190
33 Real Estate 29
34 Recreation & Entertainment 300
35 Religion & Spirituality 259
36 Science 314
37 Sports 300
38 Technology & Internet 851
39 Travel & Tourism 375
40 Video Games 300
41 Other 300


In [97]:
# write to links for octoparse
links = []
for cat in list(channels.keys())[30:]:
    for l in channels[cat]:
        links.append(f'https://in.tgstat.com/channel/{l}?category={cat}'.replace(' ', '%20'))
        
with open('data/EN_TGSTAT_LINKS/links4.txt', 'w') as f:
    f.write('\n'.join(links))

## vk2octoparse

Parse vk links to octoparse format

In [1]:
import json
import pandas as pd

CATEGORY = 'Other'

In [2]:
with open("tgparser/categoryVK.txt", 'r', encoding='utf-8-sig') as f:
    data = json.loads(f.read())
    
with open("tgparser/vkgroupsRU.json", 'r', encoding='utf-8') as f:
    groups = json.loads(f.read())

In [44]:
import urllib
import time

links = []
for query in ['архитектура', 'дизайн', 'design', 'искусство', 'architecture']:
    for i in range(10):
        OFFSET = f'&offset={25 * i}'
        QUERY = urllib.parse.quote(query)
        URL = f'https://allsocial.ru/entity?direction=1&is_closed=0&is_verified=-1&list_type=1&order_by=diff_abs&period=day&platform=1&range=1000:12000000&str={QUERY}&type_id=-1'
        response = urllib.request.urlopen(URL + OFFSET).read().decode('utf-8')
        response = json.loads(response)

        for item in response['response']['entity']:
            links.append(f'''https://vk.com/public{item['vk_id']}''')

        time.sleep(0.5)

## octoparse2json

Parse octoparse data to final data

In [42]:
LANG = 'EN'

In [43]:
import json
import pandas as pd
import os

In [54]:
data = pd.DataFrame(columns = ['Text', 'Text1', 'Field1', 'Page_URL'])
for file in os.listdir(f'data/{LANG}_TGSTAT_LINKS'):
    data = pd.concat([data, pd.read_excel(os.path.join(f'data/{LANG}_TGSTAT_LINKS', file))])
data.columns = ['title', 'description', 'recent_posts', 'category']

print(data.isna().sum())
data = data[data['category'].notna()]

title              1
description     8145
recent_posts    2769
category          16
dtype: int64


In [55]:
data['category'] = data['category'].apply(lambda x: x.split('?category=')[-1].replace('%20', ' '))
data = data[~data['category'].apply(lambda x: 'http' in x)].reset_index(drop = True)

In [65]:
import re
regex = re.compile(r'\\(?![/u"])')
for cat in data['category'].unique():
    tmp = data.query('category == @cat')
    
    for title in tmp['title'].unique():
        json_data = {}
        try:
            channel_data = tmp.query('title == @title')
            json_data["title"] = str(title).replace("'", '').replace('"', '')
            json_data["description"] = str(channel_data['description'].iloc[0]).replace("'", '').replace('"', '')
            json_data["recent_posts"] = []
            for i in range(channel_data.shape[0]):
                post = str(channel_data.iloc[i, 2]).replace("'", '').replace('"', '')
                json_data["recent_posts"].append(post)
        except:
            print('parse error')
    
        json_data = regex.sub(r"\\\\", str(json_data).replace("'", '"'))
        json_data = regex.sub(r"\\", json_data)
        with open(f'data/{LANG}_TGSTAT_DATA/{cat}.txt', 'a+', encoding='utf-8') as f:
            #json.dump(json_data, f, ensure_ascii=False)
            f.write(json_data+'\n')

parse error


In [66]:
from tqdm import tqdm_notebook

with open(f'data/{LANG}_TGSTAT_DATA/Bets & Gambling.txt', 'r', encoding='utf-8') as f:
    input_lines = f.read().split('\n')    

data = {
    "title": [],
    "description": [],
    "recent_posts": [],
}

for line in input_lines:
    try:
        line = json.loads(line.replace('\\', ''))
    except:
        pass
    
    if line != '':
        data['title'].append(line['title'])
        data['description'].append(line['description'])
        data['recent_posts'].append('\n'.join(line['recent_posts']))
        
        
data = pd.DataFrame(data)

## json2translate

Parse final data into tsv for translation

In [72]:
SEP = '<-------------новоесообщение------------->'

In [84]:
from tqdm import tqdm_notebook
LANG = 'EN'

for cat in os.listdir(f'data/{LANG}_TGSTAT_DATA'):
    data = {
        "title": [],
        "description": [],
        "recent_posts": [],
    }
    with open(f'data/{LANG}_TGSTAT_DATA/{cat}', 'r', encoding='utf-8') as f:
        input_lines = f.read().split('\n')    
        
    for line in input_lines:
        try:
            line = json.loads(line.replace('\\', ''))
            if line != '':
                
                title, desc, posts = line['title'], line['description'], SEP.join(line['recent_posts'])
                
                data['title'].append(title)
                data['description'].append(desc)
                data['recent_posts'].append(posts)
        except:
            print(line)
                
    data = pd.DataFrame(data)
    cat = cat.split('.')[0]
    data.to_excel(f'Translate/{cat}.xlsx', index = None)
    with open(f'Translate/EN-RU/{cat}.tsv', 'w+') as f:
        f.write('')

{'title': 'nan'}



C:\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:912: UserWarning: Ignoring URL 'https://twitter.com/cfkargentina/status/1356663329733902341?s=21%3c-------------новоесообщение-------------%3ehttps://twitter.com/cfkargentina/status/1353380755779407872?s=21%3c-------------новоесообщение-------------%3ehttps://twitter.com/cfkargentina/status/1345852156172259329?s=21%3c-------------новоесообщение-------------%3enan%3c-------------новоесообщение-------------%3eSin%20lugar%20a%20dudas%20este%20año%20que%20termina%20fue%20muy%20difícil%20para%20la%20mayoría%20de%20los%20argentinos%20y%20las%20argentinas.%20A%20esa%20verdadera%20pandemia%20económica%20y%20social%20que%20representó%20el%20último%20ciclo%20neoliberal%20del%20gobierno%20macrista%20se%20sumó%20este%20año,%20de%20manera%20imprevisible,%20la%20pandemia%20del%20coronavirus.Con%20la%20llegada%20de%20las%20primeras%20vacunas%20se%20abre%20una%20esperanza,%20pero%20tenemos%20que%20seguir%20cuidándonos.Por%20eso%20les%20quiero%20pe

C:\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:912: UserWarning: Ignoring URL 'https://thetruereporter.com/double-standard-biden-vowed-to-fight-like-hell-in-2019/%3c-------------новоесообщение-------------%3ehttps://thetruereporter.com/deaths-of-elderly-who-recovered-from-covid-19-but-died-after-vaccine-raise-questions/%3c-------------новоесообщение-------------%3ehttps://thetruereporter.com/corrupt-pelosis-husband-put-1m-on-tesla-right-before-biden-announced-fleet-to-turn-electric/%3c-------------новоесообщение-------------%3eMore%20at:%20https://t.me/TheTrueReporter%3c-------------новоесообщение-------------%3ehttps://thetruereporter.com/maxine-waters-now-claims-she-never-encouraged-violence-against-trump-supporters-video/%3c-------------новоесообщение-------------%3ehttps://thetruereporter.com/a-gallup-poll-says-66-of-americans-are-not-satisfied-with-the-biden-administrations-vaccine-rollout/%3c-------------новоесообщение-------------%3ehttps://thetruereporter.com/can-anti-f

C:\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:912: UserWarning: Ignoring URL 'https://halove23.blogspot.com/2021/02/windows-installer-file-read-0day_12.html%20%20%20%20%20%20%20%20%20%20%20%20nn%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20n%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20n%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20n%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20nn%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20n%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20Windows%20Installer%20File%20Read%2

C:\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:912: UserWarning: Ignoring URL 'https://youtu.be/i7gCWx5GG4A            nn                            n                    n                        n                            nn                                                                    n                                                                                    JEE Math Complete Formula Test | Menti Quiz | JEE Main 2021/2022 | Vedantu                                                                            n                                n                                                                                                            JEE Main Maths Formulas Menti Quiz By Vedantu. JEE Main Maths Formulas Complete Formula Test By Neha Agrawal Maam. Maths Complete Formulas Menti Quiz By Vedantu Math. This jee main maths formulas menti quiz session will increase your marks in JEE Main 2021 Exam. JEE Main Maths Formulas | Maths Important Formulas For JEE

C:\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:912: UserWarning: Ignoring URL 'https://youtu.be/CWbgL26zQgQ%20%20%20%20%20%20%20%20%20%20%20%20nn%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20n%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20n%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20n%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20nn%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20n%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20How%20To%20Install%20Ipa%20file%20in%20IOS%20Device%20with%20PC%20/%20Laptop%20%20%20

C:\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:912: UserWarning: Ignoring URL 'https://russia-instagram.tumblr.com/post/614586863717679104via%20A%20Feed%20Called%20by%20@RichRussianLuxuryLife%3c-------------новоесообщение-------------%3ehttps://russia-instagram.tumblr.com/post/614586844032745472via%20A%20Feed%20Called%20by%20@RichRussianLuxuryLife%3c-------------новоесообщение-------------%3ehttps://russia-instagram.tumblr.com/post/614586817731346432via%20A%20Feed%20Called%20by%20@RichRussianLuxuryLife%3c-------------новоесообщение-------------%3ehttps://russia-instagram.tumblr.com/post/614586669271777280via%20A%20Feed%20Called%20by%20@RichRussianLuxuryLife%3c-------------новоесообщение-------------%3ehttps://russia-instagram.tumblr.com/post/614586602690428928via%20A%20Feed%20Called%20by%20@RichRussianLuxuryLife%3c-------------новоесообщение-------------%3ehttps://russia-instagram.tumblr.com/post/614586582639443968via%20A%20Feed%20Called%20by%20@RichRussianLuxuryLife%3c--------

In [34]:
import os
import pandas as pd
cats = [cat for cat in os.listdir(f'data/RU_TGSTAT_DATA')]
pd.read_csv(f'''Translate/RU-EN/{cats[26].split('.')[0]}.tsv''', sep='\t')

,title,description,recent_posts
0,Broken soul | Psychology,NaN,Straightforwardness Women seek to enjoy commun...
1,Stronger Than Himself | Self-development,This is where you become a strong personality....,"❗️The combination no longer works, the guys we..."
2,Crypto BLOG,NaN,⚔️ How to become a member of the WallStreetBet...
3,❗️ CRAZY FACTS❗️,Channel about the craziest facts and discoveri...,🇬🇷 The island of Zakynthos (Greece) is a magic...
4,𝘽𝙤𝙤𝙠𝙨 | 𝙂𝙪𝙞𝙙𝙚𝙨📚,📚Checklists 📚Guides📚Books📚Self DevelopmentComm...,A bricklayer Simon dies in a small Armenian to...
...,...,...,...
159,Dmitry Chevychalov | Blog,"I travel the world, create and develop busines...","u200bu200b🙏Help, I lost my job, got into debt,..."
160,Google Переводчик,NaN,NaN
161,Исходный текст,NaN,NaN
162,ПрямолинейностьЖенщины стремятся получить от о...,NaN,NaN


## translate2json

Parse translated data back to json

In [92]:
SEP_EN = '<-------------newpost------------->'
SEP_RU = '<-------------новоесообщение------------->'

In [95]:
import os
import re
import pandas as pd

FOLDER = 'EN_TGSTAT_DATA_TRANSLATE'
regex = re.compile(r'\\(?![/u"])')
for cat in os.listdir(f'data/EN_TGSTAT_DATA'):
    cat = cat.split(".")[0]
    try:
        data = pd.read_csv(f'''Translate/RU-EN/{cat}.tsv''', sep='\t')
    except:
        continue
    data.columns = ['title', 'description', 'recent_posts']
    
    for i in range(data.shape[0]):
        channel_data = data.iloc[i]
        json_data = {}
        json_data["title"] = str(channel_data['title']).replace("'", '').replace('"', '')
        json_data["description"] = str(channel_data['description']).replace("'", '').replace('"', '')
        json_data["recent_posts"] = str(channel_data['recent_posts']).replace("'", '').replace('"', '').split(SEP_RU)
    
        json_data = regex.sub(r"\\\\", str(json_data).replace("'", '"'))
        json_data = regex.sub(r"\\", json_data)
        with open(f'data/{FOLDER}/{cat}.txt', 'a+', encoding='utf-8') as f:
            f.write(json_data+'\n')